In [28]:
import scipy.io
import numpy as np
import pandas as pd


file_path = "S1_E1_A1.mat"
mat_data = scipy.io.loadmat(file_path)
mat_data.keys()


dict_keys(['__header__', '__version__', '__globals__', 'emg', 'acc', 'stimulus', 'glove', 'subject', 'exercise', 'repetition', 'restimulus', 'rerepetition', 'age', 'circumference', 'frequency', 'gender', 'height', 'weight', 'laterality', 'sensor'])

In [58]:
import numpy as np
from nina_funcs import get_data, normalise, filter_data, windowing, get_categorical
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing

# Definição dos diretórios
base_dir = r"C:\Users\molinaa\Downloads\DB5"
output_dir = r"C:\Users\molinaa\Downloads\DB5_jan"

# Definição das repetições de treino e teste
train_reps = [1, 3, 4, 6]
test_reps = [2, 5]

# Parâmetros da janela
win_len = 40
win_stride = 20

# Mapeamento dos gestos por exercício
gestures_dict = {
    "1": [1, 2, 3, 4, 5, 6 ,7 ,8 , 9, 10],
}

def preprocess_and_save_data(file):
    """Processa e salva os dados de um arquivo .mat específico."""
    try:
        # Extração dos IDs do sujeito e do exercício
        parts = file.split('_')
        subject_id = parts[0][1:]  # "S1" -> "1"
        exercise_id = parts[1][1]  # "E1" -> "1"

        # Verificar se o exercício é válido no dicionário
        if exercise_id not in gestures_dict:
            return

        gestures = gestures_dict[exercise_id]

        # Carregar os dados
        data = get_data(base_dir, file)

        # Normalização e filtragem
        data = normalise(data, train_reps)
        emg_band = filter_data(data=data, f=(2, 50), butterworth_order=4, btype='bandpass')

        # Janelamento
        X_train, y_train, _ = windowing(emg_band, train_reps, gestures, win_len, win_stride)
        X_test, y_test, _ = windowing(emg_band, test_reps, gestures, win_len, win_stride)

        # Codificação categórica
        y_train = get_categorical(y_train)
        y_test = get_categorical(y_test)
        
        # Converter one-hot encoding para rótulos inteiros, se necessário
        if y_train.ndim > 1:
            y_train = np.argmax(y_train, axis=1)
        if y_test.ndim > 1:
            y_test = np.argmax(y_test, axis=1)

        # Salvar os arquivos processados
        np.save(os.path.join(output_dir, f"S{subject_id}E{exercise_id}_X_train.npy"), X_train)
        np.save(os.path.join(output_dir, f"S{subject_id}E{exercise_id}_y_train.npy"), y_train)
        np.save(os.path.join(output_dir, f"S{subject_id}E{exercise_id}_X_test.npy"), X_test)
        np.save(os.path.join(output_dir, f"S{subject_id}E{exercise_id}_y_test.npy"), y_test)
        
        print(f"Processamento concluído para sujeito {subject_id}, exercício {exercise_id}")
    
    except Exception as e:
        print(f"Erro no processamento do arquivo {file}: {e}")

def process_individual_files():
    tasks = []
    print("Iniciando processamento dos arquivos .mat...")

    with ThreadPoolExecutor() as executor:  # Substituindo ProcessPoolExecutor
        for file in os.listdir(base_dir):
            if file.endswith(".mat"):
                print(f"Arquivo encontrado: {file}")
                tasks.append(executor.submit(preprocess_and_save_data, file))

        # Aguardar a conclusão de todas as tarefas
        for future in as_completed(tasks):
            future.result()


if __name__ == '__main__':
    print("Iniciando processamento paralelo...")
    process_individual_files()
    print("Processamento paralelo concluído e arquivos salvos.")

Iniciando processamento paralelo...
Iniciando processamento dos arquivos .mat...
Arquivo encontrado: S10_E1_A1.mat
Arquivo encontrado: S1_E1_A1.mat
Arquivo encontrado: S2_E1_A1.mat
Arquivo encontrado: S3_E1_A1.mat
Arquivo encontrado: S4_E1_A1.mat
Arquivo encontrado: S5_E1_A1.mat
Arquivo encontrado: S6_E1_A1.mat
Arquivo encontrado: S7_E1_A1.mat
Arquivo encontrado: S8_E1_A1.mat
Arquivo encontrado: S9_E1_A1.mat
Processamento concluído para sujeito 5, exercício 1
Processamento concluído para sujeito 1, exercício 1
Processamento concluído para sujeito 3, exercício 1
Processamento concluído para sujeito 9, exercício 1
Processamento concluído para sujeito 8, exercício 1
Processamento concluído para sujeito 7, exercício 1
Processamento concluído para sujeito 4, exercício 1
Processamento concluído para sujeito 10, exercício 1
Processamento concluído para sujeito 6, exercício 1
Processamento concluído para sujeito 2, exercício 1
Processamento paralelo concluído e arquivos salvos.


In [ ]:
import numpy as np
import scipy.signal as signal
import os
import pywt

# Diretório onde os arquivos janelados foram salvos
input_dir = r"C:\Users\molinaa\Downloads\DB5_jan"
output_dir = r"C:\Users\molinaa\Downloads\DB5_feats"

# Função para extração de features de uma única janela de EMG
def extract_features(emg_window, sampling_rate=200):
    features = []
    for channel in range(emg_window.shape[1]):
        signal_data = emg_window[:, channel]

        # Domínio Temporal
        mean_abs = np.mean(np.abs(signal_data))
        rms = np.sqrt(np.mean(signal_data ** 2))
        variance = np.var(signal_data)
        wl = np.sum(np.abs(np.diff(signal_data)))  # Waveform Length
        #ssc = np.sum(np.diff(signal_data, 2) > 0)  # Slope Sign Change
        zc = np.sum(np.diff(np.sign(signal_data)) != 0)  # Zero Crossings

        # Domínio Espectral
        f, psd = signal.welch(signal_data, fs=sampling_rate, nperseg=len(signal_data))
        spectral_entropy = -np.sum(psd * np.log(psd + 1e-12)) 
        mean_freq = np.sum(f * psd) / np.sum(psd)  # Frequência Média

        coeffs = pywt.wavedec(signal_data, 'db7', level=3)
        mdwt_energy = [np.sum(np.square(c)) for c in coeffs]
        
        # Adiciona todas as features à lista
        features.extend([rms,spectral_entropy] + mdwt_energy)
    return features

# Função para aplicar extração de features em todas as janelas de um dataset
def extract_features_from_dataset(X_windows):
    return np.array([extract_features(window) for window in X_windows])

# Processar arquivos janelados e salvar features
def process_extracted_windows():
    for subject_id in range(1, 11):  # Indo do S1 ao S10
        for exercise_id in range(1, 2):  # Assumindo que há apenas E1 para cada sujeito
            try:
                # Carregar os arquivos janelados
                X_train = np.load(os.path.join(input_dir, f"S{subject_id}E{exercise_id}_X_train.npy"))
                X_test = np.load(os.path.join(input_dir, f"S{subject_id}E{exercise_id}_X_test.npy"))

                # Extrair features
                X_train_feats = extract_features_from_dataset(X_train)
                X_test_feats = extract_features_from_dataset(X_test)

                
                np.save(os.path.join(output_dir, f"S{subject_id}E{exercise_id}_X_train_feats.npy"), X_train_feats)
                np.save(os.path.join(output_dir, f"S{subject_id}E{exercise_id}_X_test_feats.npy"), X_test_feats)

                print(f"Features extraídas e salvas para S{subject_id}E{exercise_id}")
            except Exception as e:
                print(f"Erro ao processar S{subject_id}E{exercise_id}: {e}")

if __name__ == "__main__":
    print("Iniciando extração de features para arquivos janelados...")
    process_extracted_windows()
    print("Extração de features concluída.")

Iniciando extração de features para arquivos janelados...


c:\Users\molinaa\Desktop\Projeto_Estagio\clean_venv\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Features extraídas e salvas para S1E1
Features extraídas e salvas para S2E1
Features extraídas e salvas para S3E1
Features extraídas e salvas para S4E1
Features extraídas e salvas para S5E1
Features extraídas e salvas para S6E1
Features extraídas e salvas para S7E1
Features extraídas e salvas para S8E1
Features extraídas e salvas para S9E1
Features extraídas e salvas para S10E1
Extração de features concluída.


In [1]:
import numpy as np
import scipy.signal as signal
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Diretório onde os arquivos de features foram salvos
features_train_path = r"C:\Users\PC\Desktop\TCC_2025\App\Data\db2_features\features_train.npz"
features_test_path = r"C:\Users\PC\Desktop\TCC_2025\App\Data\db2_features\features_test.npz"


# Opção de treinamento: "individual" para treinar um modelo por sujeito, "global" para treinar com todos os sujeitos
train_mode = "global"  # Alterar para "global" se desejar treinar um único modelo

# Função para carregar os dados por sujeito
def load_feature_data(subject_id):
    try:
        train_data = np.load(features_train_path)
        test_data = np.load(features_test_path)

        X_train, y_train = train_data["X"], train_data["y"]
        X_test, y_test = test_data["X"], test_data["y"]
        return X_train, X_test, y_train, y_test
    except Exception as e:
        print(f"Erro ao carregar dados de S{subject_id}: {e}")
        return None, None, None, None


models = {
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "SVM": SVC(kernel='rbf', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=10),
    "LR": LogisticRegression(),
    "XGBoost": GradientBoostingClassifier()
}

subject_accuracies = []

if train_mode == "individual":
    for subject_id in range(1, 11):
        X_train, X_test, y_train, y_test = load_feature_data(subject_id)
        if X_train is None:
            continue

        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        for name, model in models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            acc = accuracy_score(y_test, y_pred)
            subject_accuracies.append({"Sujeito": f"S{subject_id}", "Modelo": name, "Acurácia": acc})

elif train_mode == "global":
    X_train_all, X_test_all, y_train_all, y_test_all = [], [], [], []
    for subject_id in range(1, 41):
        print("Sujeito:", subject_id)
        X_train, X_test, y_train, y_test = load_feature_data(subject_id)
        if X_train is None:
            continue
        X_train_all.append(X_train)
        X_test_all.append(X_test)
        y_train_all.append(y_train)
        y_test_all.append(y_test)
    
    X_train_all = np.vstack(X_train_all)
    X_test_all = np.vstack(X_test_all)
    y_train_all = np.hstack(y_train_all)
    y_test_all = np.hstack(y_test_all)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_all)
    X_test_scaled = scaler.transform(X_test_all)

    for name, model in models.items():
        print("Treinando:", name)
        model.fit(X_train_scaled, y_train_all)
        
        print("Prevendo:", name)
        y_pred = model.predict(X_test_scaled)
        
        acc = accuracy_score(y_test_all, y_pred)
        subject_accuracies.append({"Sujeito": "Global", "Modelo": name, "Acurácia": acc})

# Converter resultados para DataFrame e visualizar
import pandas as pd
subject_accuracies_df = pd.DataFrame(subject_accuracies)

plt.figure(figsize=(10, 6))
sns.barplot(x="Sujeito", y="Acurácia", hue="Modelo", data=subject_accuracies_df)
plt.title("Acurácia por Sujeito e Modelo")
plt.xlabel("Sujeito")
plt.ylabel("Acurácia")
plt.ylim(0, 1)
plt.legend(title="Modelo")
plt.xticks(rotation=45)
plt.show()

Sujeito: 1
Sujeito: 2
Sujeito: 3
Sujeito: 4
Sujeito: 5
Sujeito: 6
Sujeito: 7
Sujeito: 8
Sujeito: 9
Sujeito: 10
Sujeito: 11
Sujeito: 12
Sujeito: 13
Sujeito: 14
Sujeito: 15
Sujeito: 16
Sujeito: 17
Sujeito: 18
Sujeito: 19
Sujeito: 20
Sujeito: 21
Sujeito: 22
Sujeito: 23
Sujeito: 24
Sujeito: 25
Sujeito: 26
Sujeito: 27
Sujeito: 28
Sujeito: 29
Sujeito: 30
Sujeito: 31
Sujeito: 32
Sujeito: 33
Sujeito: 34
Sujeito: 35
Sujeito: 36
Sujeito: 37
Sujeito: 38
Sujeito: 39
Sujeito: 40
Treinando: Random Forest
Prevendo: Random Forest
Treinando: SVM


: 

In [7]:
subject_accuracies_df

,Sujeito,Modelo,Acurácia
0,Global,Random Forest,0.857159
1,Global,SVM,0.821155
2,Global,KNN,0.769597
3,Global,LR,0.507973
4,Global,XGBoost,0.667124
